In [2]:
print("hii")

hii


In [3]:
print("sohaib")

sohaib


In [4]:
print("hello world")

hello world


In [5]:
%pwd

'C:\\Users\\Welcome\\Desktop\\medi-chatbot-'

In [5]:
import os

# Use os.chdir() to change the directory
os.chdir(r"C:\Users\Welcome\Desktop\medi-chatbot-")

# Verify that the directory has been changed
print("Current Directory:", os.getcwd())


Current Directory: C:\Users\Welcome\Desktop\medi-chatbot-


In [6]:
%pwd

'C:\\Users\\Welcome\\Desktop\\medi-chatbot-'

In [26]:
from langchain.document_loaders import PyPDFLoader , DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [27]:
#Extract Data from the PDF file
def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob = "*.pdf",
                             loader_cls=PyPDFLoader)
    
    documents = loader.load()
    return documents

In [28]:
extracted_data = load_pdf_file(data = 'Data/')

In [29]:
# extracted_data


In [30]:
#chunking operation
# split the data into text chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap = 20) 
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [31]:
text_chunks = text_split(extracted_data)
print("Length of text chunks", len(text_chunks))

Length of text chunks 5860


In [32]:
from langchain.embeddings import HuggingFaceEmbeddings

In [33]:
#download the embeddings from the hugging face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings
embeddings = download_hugging_face_embeddings()

In [34]:
#cheking here embedding hua download huahai ya nhi and checking their dimension
query_result = embeddings.embed_query("hello world")
print("Length", len(query_result))


Length 384


In [36]:
#checking vector
# query_result

In [37]:
from dotenv import load_dotenv
load_dotenv()

True

In [38]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.environ.get('HUGGINGFACEHUB_API_TOKEN')


In [39]:
#initialize pinecone
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "medi-chatbot2"

pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [40]:
# Set your Hugging Face API key and pinecone 
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

In [41]:
# Embed each chunk and upsert the embeddings into your pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

MaxRetryError: HTTPSConnectionPool(host='medi-chatbot2-bw1bs4h.svc.aped-4627-b74a.pinecone.io', port=443): Max retries exceeded with url: /vectors/upsert (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000002196E07BE80>: Failed to resolve 'medi-chatbot2-bw1bs4h.svc.aped-4627-b74a.pinecone.io' ([Errno 11001] getaddrinfo failed)"))

In [43]:
#Load Existing index
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings,
)

In [44]:
docsearch

In [46]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [47]:
retrieved_docs = retriever.invoke("what is fever")

In [49]:
retrieved_docs
#this dont give the complete answer that's why we need to llm 

[Document(id='94242930-101a-478e-8cfd-95b4cdcc9e1a', metadata={'page': 69.0, 'source': 'Data\\Medical_book.pdf'}, page_content='fever in children. This disease is most often caused by\ntypes 3 and 7. Symptoms, which appear suddenly and\nusually disappear in less than a week, include:\n• inflammation of the lining of the eyelid (conjunctivitis)\n•f e v e r\n• sore throat (pharyngitis)\n• runny nose\n• inflammation of lymph glands in the neck (cervical\nadenitis)\nGALE ENCYCLOPEDIA OF MEDICINE 256\nAdenovirus infections\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 56'),
 Document(id='5fa1169f-1d39-42db-82ae-5dae54831d06', metadata={'page': 60.0, 'source': 'Data\\Medical_book.pdf'}, page_content='(38°–40°C). In addition, a general ill feeling, muscle\naches,headache, chills, and loss of appetite may be felt.\nDiagnosis\nIf lymphangitis is suspected, the person should call\nhis or her doctor immediately or go to an emergency\nroom. Acute lymphangitis could be diagnosed by the\nfamily do

In [50]:
from langchain import HuggingFaceHub
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.llms import HuggingFaceHub
from langchain_huggingface import ChatHuggingFace
from langchain.llms import HuggingFaceHub
from langchain.prompts import ChatPromptTemplate




# Initialize the Hugging Face LLM
# Initialize LLM

# Hugging Face Hub ke liye free model configuration
# llm = HuggingFaceHub(
#     repo_id="google/flan-t5-large",
#     model_kwargs={
#         "temperature": 0.7,
#         "max_new_tokens": 250,
#         "top_p": 0.9
#     }
# )


# # System prompt
# system_prompt = (
#     "You are a highly knowledgeable assistant tasked with providing detailed and accurate answers. "
#     "For the given query, please answer in detail, covering all aspects of the topic. "
#     "If the query asks for multiple components (e.g., causes, symptoms, treatments), provide a thorough breakdown for each. "
#     "Be as detailed as possible within the token limit, and ensure that you give examples where applicable. "
#     "Always reference the retrieved context when answering, and if more information is needed, request clarification politely. "
#     "If the information in the context is insufficient, clearly state that and ask for further details."
#     "\n\n{context}"
# )

# prompt = ChatPromptTemplate.from_messages(
#     [
#         ("system", system_prompt),
#         ("human", "{input}")
#     ]
# )

from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate

# Load BioGPT model and tokenizer
model_name = "BIOS/biogpt"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Adjust the system prompt for BioGPT
system_prompt = (
    "You are a highly knowledgeable medical assistant tasked with providing detailed and accurate medical answers. "
    "For the given query, please answer in detail, covering all aspects such as symptoms, causes, diagnosis, and treatments. "
    "If the query requires multiple components (e.g., treatment options, risks, outcomes), provide a thorough breakdown for each. "
    "If additional details are required, request clarification politely. "
    "Always base your responses on medical knowledge and clearly state if certain information is unavailable or requires further input.\n\n{context}"
)

# Prompt template for queries
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}")
    ]
)

# Create an LLMChain with BioGPT
llm = LLMChain(
    llm=lambda query: {
        "output": model.generate(
            **tokenizer(query, return_tensors="pt")
        )
    },
    prompt=prompt
)

# Example query
query = "What are the symptoms and treatment options for diabetes?"
response = llm.run({"input": query})
print(response)



C:\Users\Welcome\AppData\Local\Temp\ipykernel_15592\3725394950.py:19: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm = HuggingFaceHub(


In [51]:
# Create a chain for generating responses
question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriever,question_answer_chain)


In [11]:
response = rag_chain.invoke({"input": "early cancer?"})
print(response["answer"])


NameError: name 'rag_chain' is not defined